In [1]:
import numpy as np
import pandas as pd
import openpyxl
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import re
import os
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

In [2]:
# # stephen
# input_path = r'C:\Users\stephenl\OneDrive - GC International AG\ML\orderCheck_india\input_data\\'
# output_path = r'C:\Users\stephenl\OneDrive - GC International AG\ML\orderCheck_india\output_data\\'

In [3]:
# # Michelle
# input_path = r'C:\Users\michelle.tan\OneDrive - GC International AG\orderCheck_india\input_data\\'
# output_path = r'C:\Users\michelle.tan\OneDrive - GC International AG\orderCheck_india\output_data\\'

In [ ]:
# Jenny
input_path = r'C:\Users\jenny.loo\OneDrive - GC International AG\orderListData\data_input_india\\'
output_path = r'C:\Users\jenny.loo\OneDrive - GC International AG\orderListData\data_output_india\\'

In [4]:
# create function
def transform(i):

    txt = pd.read_csv(input_path+i,delimiter = "\t")

    def tgt(s):
        for i in s:
            return i

    # create new dataframe

    df = pd.DataFrame(columns = ['productcode','qty','unitprice','total','po_num'])

    

    list1=[]
    for i in txt['TextField69']:
        if re.search(r'(\d+.\d\d\d$)',i): # function to remove decimal point from thousands
            list1.append(i.replace('.',''))
        elif re.search(r',\d{3}.\d\d$',i): # remove comma from thousand
            list1.append(i.replace(',',''))
        else:
            list1.append(i)
            
    txt['TextField69']=list1
            


    # iterate thru the txt

    h=0
    for m in range(len(txt)-1):
        t = tgt(txt.loc[m])
    #     print(f"capture {t}")
        t2 = tgt(txt.loc[m+1])
        # extract po number
        if re.search(r'^\d{3}/GCC',t):
            a = t
        # extract Quantity for non floating numbers
        if re.search(r'Quantity',t) and re.search(r'^\d+$',t2):
            df.loc[h,'qty']=t2
        # extract Unitprice for floating numbers only
        if re.search(r'Unitprice',t) and re.search(r'^\d+.\d\d$',t2):
            df.loc[h,'unitprice']=t2
        if re.search(r'Material',t) and re.search(r'^(\d(\s+)?){6}$',t2):
            df.loc[h,'productcode']=t2
            df.loc[h,'po_num']=a
        h+=1
        
    dftotal = pd.DataFrame(columns = ['productcode','po_num','qty','unitprice','total'])
    df1 = df[['productcode']].dropna().reset_index(drop=True)
    df4 = df[['po_num']].dropna().reset_index(drop=True)
    df2 = df[['qty']].dropna().reset_index(drop=True)
    df3 = df[['unitprice']].dropna().reset_index(drop=True)
    
    dftotal = pd.concat([df1,df4,df2,df3], axis=1)
    
    # convert all to numbers
    dftotal.qty = [int(i) for i in dftotal.qty]
    dftotal.unitprice = [float(i) for i in dftotal.unitprice]
    
    # fill in total column
    dftotal['total']=dftotal.qty * dftotal.unitprice
    
    dftotal = dftotal.reset_index(drop=True)
        
    return dftotal

# return today's date
def today():
    return datetime.now().strftime('%d-%b-%y_%I_%M_%S_%p')

In [5]:
# change directory to input path
os.chdir(input_path)

# ensure each document calculation is correct
def checknum(f):
    list2 = f.count().tolist()
    d = list2[0]
    return any(i == d for i in list2)

# concatenate all invoices together

os.chdir(input_path)
os.listdir(input_path)
df2 = pd.DataFrame()
for i in os.listdir(input_path):
    file,ext = os.path.splitext(i)
    if re.search(r'^INV',i) and ext=='.txt':
        df3 = transform(i)
        # condition: df3 must not be empty, all column lengths must be equal
        if len(df3)>0 and checknum(df3):
            df2=pd.concat([df2,df3])
        else:
            with open(output_path+f"Alert_{i}_.txt", "w") as f:
                f.write(f"{i} needs fixing, and will be excluded from the computation results.")
            print(f'Oops! {i} needs fixing, and will be excluded from the computation results.')
            continue
    else:
        continue
df2 = df2.reset_index(drop=True)

In [6]:
# make a copy of df2 and assign to df6
df6=df2.copy()

In [7]:
# rename productcode to productcode_6

df2.rename(columns={'productcode': 'productcode_6'}, inplace=True)

# trim the columns
df2 = df2[['productcode_6','po_num','qty']]

## Read master india

In [8]:
# read master record
ms = pd.read_excel(input_path+'master_india.xlsx', sheet_name='INR Price list to GCIndia 0421 ')

In [9]:
ms=(ms[['Global Code','Column1','Description','New price from GC Asia to GC India (S$)                 As from Apr 2017',
       'INR price from GC Asia to GC India effective April 2021 orders \n(exchange rate 52.20)']])

In [10]:
ms.columns = [' '.join(i.split()) for i in ms.columns]

In [11]:
ms.rename(columns={'Global Code':'productcode_6','Column1':'productcode_12','Description':'description',
                  'New price from GC Asia to GC India (S$) As from Apr 2017':'sgd_price',
                  'INR price from GC Asia to GC India effective April 2021 orders (exchange rate 52.20)':'inr_price'},
         inplace=True)

In [12]:
# convert productcode_6 to str

ms.productcode_6 = ms.productcode_6.astype(str)

In [13]:
# product leading zeros

leadingzeros = (lambda x: '0'+x[:-2] if len(x)==7 else '00'+x[:-2] if len(x)==6 else '000'+x[:-2] if len(x)==5 else
               '0000'+x[:-2] if len(x)==4 else '00000'+x[:-2] if len(x)==3 else '000000'+x[:-2] if len(x)==2 else 'na')

In [14]:
ms.productcode_6 = [leadingzeros(i) for i in ms.productcode_6]

In [15]:
ms=ms.dropna()

## Merge both files

In [18]:
new= df2.merge(ms, left_on='productcode_6',right_on='productcode_6')

In [19]:
new['total_sgd']=new.qty * new.sgd_price
new['total_inr']=new.qty * new.inr_price
new['price_source']='master'

In [20]:
# # rename inr_price to merge with df9
# new1 = new.rename(columns={'inr_price':'inr_price2'})

new.head()
new = (new[['productcode_6','productcode_12','po_num','description','qty','sgd_price','total_sgd','inr_price',
           'total_inr','price_source']])

In [21]:
# # save to excel
# new.to_excel(output_path+'tax_invoice.xlsx', index=False)
h=today()
new.to_excel(output_path+f'india_{h}.xlsx',index=False)

In [31]:
new1=new.groupby(['productcode_6','productcode_12','po_num','description','sgd_price','inr_price','price_source']).\
agg(qty=('qty','sum')).reset_index()

In [32]:
new1['total_sgd']=new1.sgd_price*new1.qty
new1['total_inr']=new1.inr_price*new1.qty

In [33]:
new1 = new1[['productcode_6','productcode_12','po_num','description','qty','sgd_price','total_sgd','inr_price',\
            'total_inr','price_source']]

## compare price 

In [34]:
# prepare invoice file for comparison by trimming columns

df6.rename(columns={'productcode':'productcode_6'},inplace=True)

inv = df6.merge(ms, left_on='productcode_6', right_on='productcode_6')

inv['price_source']='invoice'

inv = inv[['productcode_6','productcode_12','unitprice','price_source']]

inv.rename(columns={'unitprice':'inr_price'},inplace=True)


In [35]:
# preparing master file for price comparison and assign to mas

mas = new1[['productcode_6','productcode_12','inr_price','price_source']]

In [36]:
# compare inv and mas

x = pd.concat([inv,mas])
compare = x.drop_duplicates(subset=['productcode_6','inr_price'],keep=False)

In [37]:
# # save to excel
# df5.to_excel(output_path+'tax_invoice.xlsx', index=False)
h=today()
compare.to_excel(output_path+f'india_price_compare_{h}.xlsx',index=False)